<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/tapi-logo-small.png" />

This notebook free for educational reuse under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

Created by J.D. Porter  for the 2024 Text Analysis Pedagogy Institute, with support from [Constellate](https://constellate.org).
<br />
____

# `Small Language Models` `2`

This is lesson `2` of 3 in the educational series on `small language models`. This notebook is intended `to teach methods for finding key words in context (KWIC), moving from there to counting collocates, and restructuring collocate data as an array`. 

**Skills:** 
* Text analysis
* Python

**Audience:** `Teachers` / `Learners` / `Researchers`

**Use case:** `Tutorial`

**Difficulty:** `Beginner`

**Completion time:** `90 minutes`

**Knowledge Required:** 
```
* Python basics (variables, functions, lists, dictionaries)
```

**Knowledge Recommended:**
```
n/a
```

**Learning Objectives:**
After this lesson, learners will be able to:
```
1. Find the context for a target term in a text
2. Count the collocates for a target term in a text
3. Structure collocate information as an array
```
**Research Pipeline:**
```
If you want to try some of this on your own, you can:
1. Gather some texts and save them on your machine as .txt files
2. Use whatever steps you're interested in from this notebook
3. Interpret!
```
___

# Required Python Libraries
 * To keep things simple, we will try to work with very few libraries in this notebook (just 'os', which you do not need to install, though you do need to import it)

In [ ]:
### Import Libraries ###
import os

# Required Data

**Data Format:** 
* plain text (.txt)

**Data Source:**
* included files (though you may supplement these whenever you feel comfortable doing so!)

**Data Quality/Bias:**

Included texts are from freely available sources like Project Gutenberg and Wikipedia. They have not been vetted for textual accuracy relative to, say, a scholarly edition.

**Data Description:**

This lesson uses textual data in .txt format (utf-8 encoding) from various sources.

# Introduction...

Here are a few ideas mentioned in Monday's session:
 * "we can capture information about the meanings of words by measuring **how they go together in texts**"
 * "if you know **how words are distributed across a big corpus** of language, you can determine their semantic properties (and possibly other components of meaning as well)"
 * "we look up **how often the words** "penguin", "duck", and "eagle" **appear near the word** "flies""

Today we'll get more specific about what it means for words to "go together", to "appear near" each other, or to be "distributed across a corpus". On a philosophical level, these ideas are rich and complex. On a practical level, though, exploring them is surprisingly simple.

We'll begin by thinking about Key Words in Context (KWIC), move on to collocates, and then end by rearranging our data into an array (if this term is less intuitive for you, you can basically think of it as a table, and you'll have the idea). The first two steps are useful in their own right; you can do a lot of productive analysis even if you stop at either. The last is really more of preparatory step for Friday. But all three help us see how we can quantify the theoretical ideas expressed above.

# KWIC


Key Words in Context, or KWIC, is a term for finding the words that surround some target term. A list of KWICs is similar to a concordance. The steps for producing KWICs are pretty straightforward:

1. Decide on a window size, *n*
2. Find every instance of a target term
3. Grab all the words *n* to the left and *n* to the right of the target term

Let's say our target term is "truth", and we have the following passage:

"It is a **truth** universally acknowledged, that a single man in possession of a good fortune must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this **truth** is so well fixed in the minds of the surrounding families, that he is considered as the rightful property of some one or other of their daughters."

If our window size is 3, we get the following KWICS:

    it is a truth universally acknowledged that
    a neighbourhood this truth is so well

Or with a window size of 10, we'd have:

    it is a truth universally acknowledged that a single man in possession of a
    man may be on his first entering a neighbourhood this truth is so well fixed in the minds of the surrounding

Not too complicated! There's a lot to decide (how big should the window be? Should we include the target term? How do we handle beginnings and endings?). But the gist is simple, and the payoff is pretty clear too: This helps us isolate a term and see something about how it's used. So let's dig in!

We'll begin by turning a text file into some words

In [ ]:
# Here are some useful functions we built on Monday

# Takes a txt filename and returns a list of its words
def file2words(somefilename,clean = True):
    with open(somefilename) as source:
        text = source.read()
    words = text.split()
    if clean:
        words = [cleanword(w) for w in words]
    return words

# Takes a word and returns a cleaned up version of it
# This works a little differently than the version from Monday
def cleanword(someword,remove_apostrophe=True):
    word = someword.casefold()
    while word and not word[0].isalpha():
        word = word[1:]
    while word and not word[-1].isalpha():
        word = word[:-1]
    return word

# Takes a count dictionary and returns a sorted list of its key,value pairs
def sortdict(somecountdict):
    s = sorted(somecountdict,key = lambda i:somecountdict[i],reverse=True)
    sorted_counts = [(word,somecountdict[word]) for word in s]
    return sorted_counts

In [ ]:
# We'll work with a small passage for now

passage = "It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered as the rightful property of some one or other of their daughters."

words = passage.split()
words = [cleanword(w) for w in words]

words[:10]

In [ ]:
# You can use 'enumerate' to keep track of where you are as you iterate through something

a = 'apple'
l = ['ant','bug','cat','dog','elk']

for n,letter in enumerate(a):
    print(n,letter)

In [ ]:
# Let's build a KWIC function together
# Remember that in Python, lists retain order!

# Takes a target term and find its KWICs in some list of words
def get_KWIC(sometargetterm,somewords,window = 3,excl_target = True):
    KWIC = []
    for n,word in enumerate(somewords):
        if word == sometargetterm:
            start = max(n-window,0)
            end = min(n+window+1,len(somewords)-1)
            if excl_target:
                k = somewords[start:n] + somewords[n+1:end]
            else:
                k = somewords[start:end]
            KWIC.append(k)
    return KWIC

In [ ]:
l[1:3]

In [ ]:
kwics = get_KWIC('is',words,window=5)
for k in kwics:
    outstr = " ".join(k)
    print(outstr)

In [ ]:
# Seeing KWICs in action in a file

fn = 'Fitzgerald_GreatGatsby.txt'

words = file2words(fn)

kwics = get_KWIC('tom',words,window=10)

# for k in kwics:
#     outstr = " ".join(k)
    # print(outstr,"\n")

# Getting KWICs for multiple terms

kwic_d = {}

target_terms = ['daisy','tom','gatsby','nick','myrtle']

for t in target_terms:
    kwic_d[t] = get_KWIC(t,words,window=5)

In [ ]:
# Seeing KWICs in action for a larger corpus

source_directory = 'jazz_bios'

# Takes a directory and returns a list of files of the specified type
def dir2files(somedir,file_type='.txt',path=True):
    files = os.listdir(source_directory)
    if path:
        files = [os.path.join(source_directory,f) for f in files]
    files = [f for f in files if f.endswith(file_type)]
    return files

files = dir2files(source_directory,file_type='.txt')

kwic_d = {}

target_terms = ['trumpet','coronet','guitar','piano','saxophone']

for f in files:
    words = file2words(f)
    for t in target_terms:
        if t not in kwic_d:
            kwic_d[t] = []
        kwics = get_KWIC(t,words,window=5)
        for k in kwics:
            kwic_d[t].append(k)

# Collocates

The **collocates** of a term are the words that appear near it in some corpus. That is, they're the words that "co-located" with the term. Yet another way to put it is that for any target term, its collocates are all of the words in its KWICs. For instance, looking at the KWICs for "truth" in our passage with window size 3:

    it is a [truth] universally acknowledged that
    a neighbourhood this [truth] is so well

We'd get the following collocates (leaving out the target term itself):

collocate | count
----|----
it | 1
is | 2
a | 2
universally | 1
acknowledged | 1
that | 1
neighborhood | 1
this | 1
so | 1
well | 1

As you can see, what counts as a collocate depends on the window size. We'd get a very different table with a window of 10 or 100 words, and in some applications people even zoom out to the document level. In that case, "rightful" would be a collocate of "uniting" in *Pride and Prejudice*, even though the former never appears again after the first two sentences, and the latter never appears until the last sentence!

Since we already have a way to find KWICs, we'll use it as a step in finding collocates. But it's worth noting that you don't *have* to go through KWICs to get to collocates. You're always collecting collocates from the window surrounding a word, but you don't necessarily need to record all of the relevant contexts—it's fine to skip straight to the counts, if that's all you want.

In [ ]:
kwics = get_KWIC('truth',words,window=3)
kwics

In [ ]:
# Let's build a collocate function together, using our existing KWICs function

# Take a list of KWICs and return a dictionary of word counts
def kwics2colls(somekwiclist):
    counts = {}
    for k in somekwiclist:
        for w in k:
            if w not in counts:
                counts[w] = 0
            counts[w] += 1
    return counts

In [ ]:
colls = kwics2colls(kwics)
colls

In [ ]:
# What about the target term itself?
sentence = "Rose is a rose is a rose is a rose."

In [ ]:
stops = file2words('stopwords.txt')
stops = [cleanword(w) for w in stops]

In [ ]:
# Running the collocate procedure on a file

fn = 'Fitzgerald_GreatGatsby.txt'

words = file2words(fn)

target_term = 'series'

kwic = get_KWIC(target_term,words,window=10)

coll_d = kwics2colls(kwic)

s = sortdict(coll_d)
s = [i for i in s if i[0] not in stops]

In [ ]:
# Working with multiple target_terms

fn = 'Fitzgerald_GreatGatsby.txt'

words = file2words(fn)

target_terms = ['daisy','tom','gatsby','nick','myrtle']

all_coll_d = {}

for t in target_terms:
    kwic = get_KWIC(t,words,window=10)
    coll_d = kwics2colls(kwic)
    all_coll_d[t] = coll_d

In [ ]:
# Looking at some of the results from Gatsby
s = sortdict(all_coll_d['nick'])
s = [i for i in s if i[0] not in stops]

In [ ]:
# Run the procedure on a corpus in a directory

all_coll_d = {}

target_terms = ['trumpet','coronet','guitar','piano','saxophone']

for f in files:
    words = file2words(f)
    for t in target_terms:
        if t not in all_coll_d:
            all_coll_d[t] = {}
        kwic = get_KWIC(t,words,window=10)
        coll_d = kwics2colls(kwic)
        for word,count in coll_d.items():
            if word not in all_coll_d[t]:
                all_coll_d[t][word] = 0
            all_coll_d[t][word] += count

In [ ]:
# Looking at some of the results from the jazz corpus
s = sortdict(all_coll_d['guitar'])
s = [i for i in s if i[0] not in stops]

# Toward an array

Let's return for a second to an example from our first session. When we were discussing the distance between various bird words, we modeled their data with a little table:

| term | "flies" | "swims" | "hockey" |
| ---- | ---- | ---- | ---- |
|penguin | 1 | 9 | 9
|duck | 8 | 7 | 8
|eagle | 10 | 1 | 0

This could also be depicted as a series of lists:

```Python
    penguin = [1,9,9]
    duck = [8,7,8]
    eagle = [10,1,0]
```

Now, all we're really showing here is data about a few collocates for each of the bird terms. But we've arranged that data a little differently than in our work in the collocates section, above. The arrangement consists of a series of **vectors**, which, as we learned on Monday, are just lists of numbers. Calculating relationships between words is much easier when we use vectors. We'll get into the details about distance on Friday, but producing the table or vector structure is an important first step.

In [ ]:
# Before we dig in, an important concept
# A list of lists works like a table (and kind of looks like one, too)

bird_array = [['term','flies','swims','hockey'],[1,9,9],[8,7,8],[10,1,0]]

for row in bird_array:
    print(row)

In [ ]:
# First, let's produce a table that captures the data for every word in our passage

words = passage.split()
words = [cleanword(w) for w in words]

uniques = set(words)

all_coll_d = {}

for term in uniques:
    kwics = get_KWIC(term,words,window=3)
    coll_d = kwics2colls(kwics)
    all_coll_d[term] = coll_d

header = ['term']

header.extend(uniques)

array = [header]

for term,data in all_coll_d.items():
    new_row = [term]
    for i in header[1:]:
        if i in data:
            new_row.append(data[i])
        else:
            new_row.append(0)
    array.append(new_row)

In [ ]:
for row in array:
    print(row[:10])

In [ ]:
# Next, let's try a table that shows how a few words relate to each other in The Great Gatsby

fn = 'Fitzgerald_GreatGatsby.txt'

words = file2words(fn)

target_terms = ['daisy','tom','gatsby','nick','myrtle']

all_coll_d = {}

for t in target_terms:
    kwic = get_KWIC(t,words,window=10)
    coll_d = kwics2colls(kwic)
    all_coll_d[t] = coll_d

array = [['term']]
array[0].extend(target_terms)

for person in target_terms:
    new_row = [person]
    for i in target_terms:
        if i in all_coll_d[person]:
            new_row.append(all_coll_d[person][i])
        else:
            new_row.append(0)
    array.append(new_row)

In [ ]:
all_coll_d['daisy']['daisy']

In [ ]:
array